## Kit Big Data - DM4

Dans ce DM, on veut explorer le lien potentiel entre la densité de médecins par spécialité et par territoire et la pratique du dépassement d'honoraires. On prendra ici une approche simple pour répondre à cette question. On va établir la densité de médecins sur le territoire (en nombre de pratiquants pour 100000 habitants) par spécialité. On calculera ensuite le taux de dépassements d'honoraires (le nombre de dépassements par rapport au nombre total d'actes), là encore par spécialité. On finira en calculant le coefficient de corrélation entre les deux, que l'on anticipe négatif si effectivement une plus faible densité de pratiquants implique d'avantage de dépassements d'honoraires.

In [1]:
# imports
import pandas as pd
import os

In [2]:
# Récupération des données par spécialité

# la table trouvée sur le site: "http://www.data.drees.sante.gouv.fr/TableViewer/tableView.aspx" n'est pas au format csv
# il faut passer par une sauvegarde csv sur disque et chargement manuel
sf = "D:\kbd\specialite.csv"
sd = pd.read_csv(sf, sep=",", encoding="ISO-8859-1")
sd.head()

,mode exercice,ensemble des modes d'exercice,Libéraux exclusifs,Mixtes,Salariés hospitaliers,Autres salariés
0,Ensemble des spécialités d'exercice,226219,103335,26423,69306,27155
1,Spécialistes,123753,43121,18567,50345,11720
2,Anatomie et cytologie pathologiques,1657,599,165,841,52
3,Anesthésie-réanimation,11497,3832,767,6454,444
4,Biologie médicale,3018,1154,64,1279,521


In [3]:
# transformations

# on commence par renommer les colonnes utiles et enlever les inutiles
sd.columns = ['specialité','total','drop1','drop2','drop3','drop4']
del sd['drop1']
del sd['drop2']
del sd['drop3']
del sd['drop4']

# on convertit en densité (on divise par la population totale française de 67,19 millions et multiplie par 100000)
sd['densité']=sd['total']/67190000*100000

# on enlève les lignes redondantes et inutiles
sd = sd.drop([0,1,42], axis=0).sort_values(['specialité']).reset_index().drop(['index'], axis=1)

# aperçu
sd.head()

,specialité,total,densité
0,Anatomie et cytologie pathologiques,1657,2.466141
1,Anesthésie-réanimation,11497,17.111177
2,Biologie médicale,3018,4.491740
3,Cardiologie et maladies vasculaires,7175,10.678672
4,Chirurgie générale,3563,5.302872


In [4]:
# créer dans sd une colonne de correspondance de code de spécialité
# cette colonne est nécessaire pour créer une nomenclature commune de spécialités avec les tables de dépassement à venir
sd['codecorr'] = [37,2,38,3,4,43,45,4,46,47,16,48,49,5,42,8,70,79,78,34,71,0,1,9,72,0,10,32,35,11,73,15,13,33,12,6,76,0,14,31,80]

# on filtre pour enlever les valeurs 0 (les spécialités qui n'ont pas de correspondance dans les tables de dépassement)
sd = sd[sd ['codecorr'] != 0].sort_values('codecorr').reset_index(drop = True)

# aperçu
sd.head()

,specialité,total,densité,codecorr
0,Médecine générale,102466,152.501860,1
1,Anesthésie-réanimation,11497,17.111177,2
2,Cardiologie et maladies vasculaires,7175,10.678672,3
3,Chirurgie générale,3563,5.302872,4
4,Chirurgie orthopédique et traumatologie,3404,5.066230,4


In [5]:
# Récupération données par dépassement

# on utilise la base de donnée DAMIR, trouvable à l'URL:
# https://www.data.gouv.fr/fr/datasets/depenses-d-assurance-maladie-hors-prestations-hospitalieres-donnees-nationales/
# on ne retient que les données pour 2018, les plus récentes
# il faut passer par une sauvegarde csv sur disque et chargement manuel pour éviter les erreurs de mémoire
df = r"D:\kbd\N201801.csv"
dd = pd.read_csv(df, sep=";",encoding = "ISO-8859-1", low_memory=False)
df = r"D:\kbd\N201802.csv"
dd = dd.append(pd.read_csv(df, sep=";",encoding = "ISO-8859-1", low_memory=False))
df = r"D:\kbd\N201803.csv"
dd = dd.append(pd.read_csv(df, sep=";",encoding = "ISO-8859-1", low_memory=False))
df = r"D:\kbd\N201804.csv"
dd = dd.append(pd.read_csv(df, sep=";",encoding = "ISO-8859-1", low_memory=False))
df = r"D:\kbd\N201805.csv"
dd = dd.append(pd.read_csv(df, sep=";",encoding = "ISO-8859-1", low_memory=False))
df = r"D:\kbd\N201806.csv"
dd = dd.append(pd.read_csv(df, sep=";",encoding = "ISO-8859-1", low_memory=False))
df = r"D:\kbd\N201807.csv"
dd = dd.append(pd.read_csv(df, sep=";",encoding = "ISO-8859-1", low_memory=False))
df = r"D:\kbd\N201808.csv"
dd = dd.append(pd.read_csv(df, sep=";",encoding = "ISO-8859-1", low_memory=False))

# aperçu
dd.head()

,l_serie,SERIE,prs_nat,l_prs_nat,rem_date,sns_date,asu_nat,l_asu_nat,cpl_cod,l_cpl_cod,...,l_exe_spe,exe_spe1,l_exe_spe1,exe_stj1,l_exe_stj1,rem_mon,rec_mon,dep_mon,act_dnb,act_coe
0,C Omnipraticiens,111111,1098,CONSULTATION CCMU 3,201801,201801,1,Maladie,0,Sans majoration,...,01-Médecine générale,11,Médecins Omnipraticiens,1,Libéral,"0,00","0,00","0,00",0.0,"0,00"
1,C Omnipraticiens,111111,1098,CONSULTATION CCMU 3,201801,201801,1,Maladie,0,Sans majoration,...,01-Médecine générale,11,Médecins Omnipraticiens,1,Libéral,"44.371,20","55.464,00","0,00",1852.0,"1.848,80"
2,C Omnipraticiens,111111,1098,CONSULTATION CCMU 3,201801,201801,1,Maladie,0,Sans majoration,...,01-Médecine générale,11,Médecins Omnipraticiens,2,Salarié,"24,00","30,00","0,00",1.0,"1,00"
3,C Omnipraticiens,111111,1098,CONSULTATION CCMU 3,201801,201801,1,Maladie,0,Sans majoration,...,01-Médecine générale,11,Médecins Omnipraticiens,1,Libéral,"204,00","240,00","0,00",8.0,"8,00"
4,C Omnipraticiens,111111,1098,CONSULTATION CCMU 3,201801,201801,1,Maladie,0,Sans majoration,...,01-Médecine générale,11,Médecins Omnipraticiens,1,Libéral,"360,00","413,79","0,00",15.0,"13,80"


In [6]:
# on commence par renommer les colonnes utiles et enlever les inutiles
dd.columns = ['drop1','drop2','drop3','drop4','drop5','drop6','drop7','drop8','drop9','drop10','drop11','code','specialite','drop12','drop13','drop14','drop15','drop16','drop17','drop18','drop19','drop20','drop21','drop22','drop23','depassement','drop24','drop25']
del dd['drop1']
del dd['drop2']
del dd['drop3']
del dd['drop4']
del dd['drop5']
del dd['drop6']
del dd['drop7']
del dd['drop8']
del dd['drop9']
del dd['drop10']
del dd['drop11']
del dd['drop12']
del dd['drop13']
del dd['drop14']
del dd['drop15']
del dd['drop16']
del dd['drop17']
del dd['drop18']
del dd['drop19']
del dd['drop20']
del dd['drop21']
del dd['drop22']
del dd['drop23']
del dd['drop24']
del dd['drop25']

# on redéfinit le type des colonnes pour pouvoir travailler avec
dd['depassement'] = dd['depassement'].str.replace('.','').str.replace(',','.')
dd['depassement'] = dd['depassement'].astype(float)

# aperçu
dd.head()

,code,specialite,depassement
0,1,01-Médecine générale,0.0
1,1,01-Médecine générale,0.0
2,99,99-Spécialité Inconnue,0.0
3,1,01-Médecine générale,0.0
4,1,01-Médecine générale,0.0


In [7]:
# on compte maintenant le nombre de dépassements, par spécialité
dd['booldepassement'] = dd['depassement']>0
numdep = dd[dd['booldepassement'] == True].groupby('code').count().reset_index()

# on efface les colonnes devenues inutiles
del numdep['specialite']
del numdep['booldepassement']

# aperçu
numdep.head()

,code,depassement
0,1,31871
1,2,6289
2,3,8825
3,4,6000
4,5,9189


In [8]:
# on établit ensuite le nombre total d'actes par specialité
numtot = dd.groupby('code').count().reset_index()
del numtot['specialite']
del numtot['booldepassement']
numtot.columns = ['code','actes']

# aperçu
numtot.head()

,code,actes
0,1,212135
1,2,44551
2,3,48275
3,4,39124
4,5,40044


In [9]:
# on utilise les deux tables précédentes pour le calcul des taux de depassement par specialité
td = pd.concat([numdep, numtot], axis=1, sort=False)
td.columns = ['drop1','depassement','code','actes']
del td['drop1']
td = td[['code', 'depassement', 'actes']]
td['tauxdepassement'] = td['depassement'] / td['actes']

# aperçu
td.head()

,code,depassement,actes,tauxdepassement
0,1,31871.0,212135,0.150239
1,2,6289.0,44551,0.141164
2,3,8825.0,48275,0.182807
3,4,6000.0,39124,0.153359
4,5,9189.0,40044,0.229473


In [10]:
# on opère finalement une jointure sur les codes de spécialités entre la table de densité et la table de taux de dépassements
final = sd.set_index('codecorr').join(td.set_index('code'))

# aperçu
final.head()

,specialité,total,densité,depassement,actes,tauxdepassement
1,Médecine générale,102466,152.501860,31871.0,212135,0.150239
2,Anesthésie-réanimation,11497,17.111177,6289.0,44551,0.141164
3,Cardiologie et maladies vasculaires,7175,10.678672,8825.0,48275,0.182807
4,Chirurgie générale,3563,5.302872,6000.0,39124,0.153359
4,Chirurgie orthopédique et traumatologie,3404,5.066230,6000.0,39124,0.153359


In [11]:
# pour conclure, on estime la correlation entre densité de spécilaistes et taux de dépassement d'honoraires
correlation = final['densité'].corr(final['tauxdepassement'])
print("La correlation entre densité et taux de dépassement est de " + str(round(correlation,3)) + ".")

La correlation entre densité et taux de dépassement est de -0.046.


### Conclusion

Il existe bien une correlation négative entre densité de pratiquants et taux de dépassement d'honoraires. La corrélation (négative) est de l'ordre de 5%, ce qui n'est pas négligeable même si elle reste modérée. La question des déserts médicaux et de l'égalité face à l'accès aux soins est donc bien justifiée.